In [2]:
import requests
import datetime, time
import pandas as pd
from pandas import DataFrame
import sqlite3
import bs4 
# user define package import
import sys
sys.path.append("/home/ubuntu/workspace/favis")
from msgbot.favisbot import favisbot
import util.krx_util as util

In [15]:
today = datetime.datetime.now().strftime('%Y%m%d')
totalpage = 2
page = 1

for i in range(1, 3):
    url = "http://dart.fss.or.kr/dsac001/mainY.do?selectDate=" + today + "&page=" + str(i)
    print(url)
    data = requests.get(url)
    data = bs4.BeautifulSoup(data.text, "html5lib")
    #articles = data.findAll("table", attrs={'class':'view'})
    notices = data.findAll("tr")
    header = notices[0]
    print(len(notices))

    datalist = []
    for i in range(1,len(notices)):
        tds = notices[i].findAll("td")
        time = tds[0].text.strip()
        link = 'http://dart.fss.or.kr' + tds[2].find('a')['href']
        title = tds[2].find('a').text.strip()

        company = tds[3].text.strip()
        regdate = tds[4].text.strip()

    #    print('%s,%s,%s,%s,%s,%s' % (today, time, title, link, company, regdate))
        datalist.append((today, time, title, link, company, regdate))

    try:
        con = sqlite3.connect('officialnotice.db')

        with con:
            cur = con.cursor()
            cur.execute('create table if not exists official_notice ( date TEXT, time TEXT, title TEXT, link TEXT, compnay TEXT, regdate TEXT)')
            #cur.execute('CREATE UNIQUE INDEX data_idx ON official_notice (date, time, link)')

            cur.executemany("INSERT OR REPLACE INTO  official_notice VALUES (?, ?, ?, ?, ?, ?)", datalist)
            con.commit()
    except sqlite3.Error as e:
        if con:
            con.rollback()
        print ("error %s" % e.args[0])

    finally:
        if con:
            con.close()        

http://dart.fss.or.kr/dsac001/mainY.do?selectDate=20160510&page=1
101
http://dart.fss.or.kr/dsac001/mainY.do?selectDate=20160510&page=2
101


In [16]:
#-*- coding: utf-8 -*-
bot = favisbot()

try:
    con = sqlite3.connect('officialnotice.db')

    with con:
        cur = con.cursor()
        cur.execute("select * from official_notice where title like '%증자%' or title like '%배정%'")

        rows = cur.fetchall()
        for row in rows:
            print ("%s|%s|%s|%s|%s|%s " %  (row[0], row[1], row[2], row[3], row[4], row[5]) )
            bot.whisper('plain','[유상증자 관련 공시]'
                        +'\n공시일시: ' + row[0]+" "+row[1]
                        +"\n회사: "+row[4]
                        +"\n공시내용: "+row[2]
                        +"\n상세링크: "+row[3])

except sqlite3.Error as e:
    if con:
        con.rollback()
    print ("error %s" % e.args[0])

finally:
    if con:
        con.close()


20160510|17:51|유상증자또는주식관련사채등의청약결과(자율공시)|http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160510800606|신성솔라에너지|2016.05.10 
20160510|16:46|[기재정정]주요사항보고서(유상증자결정)|http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160510002165|한진칼|2016.05.10 
20160510|16:31|유상증자또는주식관련사채등의발행결과(자율공시)|http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160510800475|청호컴넷|2016.05.10 
20160510|16:30|유상증자또는주식관련사채등의발행결과(자율공시)|http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160510800471|청호컴넷|2016.05.10 
